## 1-Hyperparameter Tuning

### 1-1 像sklearn一样构建模型

In [1]:
import tensorflow as tf
import numpy as np
import tensorflow.keras as keras

In [5]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)


In [2]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation='relu'))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss='mse', optimizer=optimizer)
    return model

In [6]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

keras_reg.fit(X_train, y_train, epochs=100, 
              validation_data=(X_valid, y_valid),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Train on 11610 samples, validate on 3870 samples
Epoch 1/100
11610/11610 [==============================] - 2s 172us/sample - loss: 1.3083 - val_loss: 0.5988
Epoch 2/100
11610/11610 [==============================] - 1s 108us/sample - loss: 0.5981 - val_loss: 1.6724
Epoch 3/100
11610/11610 [==============================] - 1s 110us/sample - loss: 0.5339 - val_loss: 0.7243
Epoch 4/100
11610/11610 [==============================] - 1s 103us/sample - loss: 0.4942 - val_loss: 1.1167
Epoch 5/100
11610/11610 [==============================] - 1s 109us/sample - loss: 0.4661 - val_loss: 0.6405
Epoch 6/100
11610/11610 [==============================] - 1s 112us/sample - loss: 0.4480 - val_loss: 1.0688
Epoch 7/100
11610/11610 [==============================] - 1s 115us/sample - loss: 0.4338 - val_loss: 0.4438
Epoch 8/100
11610/11610 [==============================] - 1s 111us/sample - loss: 0.4228 - val_loss: 0.6824
Epoch 9/100
11610/11610 [==============================] - 1s 109us/sample - lo

In [7]:
mse_test = keras_reg.score(X_test, y_test)

5160/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

### 1-2 随机搜索

In [9]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    'n_hidden': [0, 1, 2, 3],
    'n_neurons': np.arange(1, 100),
    'learning_rate': reciprocal(3e-4, 3e-2)
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3)
rnd_search_cv.fit(X_train, y_train, epochs=100, 
                  validation_data=(X_valid, y_valid), 
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Train on 7740 samples, validate on 3870 samples
Epoch 1/100
7740/7740 [==============================] - 1s 150us/sample - loss: 2.6320 - val_loss: 909.3442
Epoch 2/100
7740/7740 [==============================] - 1s 113us/sample - loss: 23.1533 - val_loss: 2995.4066
Epoch 3/100
7740/7740 [==============================] - 1s 112us/sample - loss: 33.2371 - val_loss: 15597.6810
Epoch 4/100
7740/7740 [==============================] - 1s 118us/sample - loss: 149.4604 - val_loss: 50594.7046
Epoch 5/100
7740/7740 [==============================] - 1s 115us/sample - loss: 2981.1940 - val_loss: 177017.6976
Epoch 6/100
7740/7740 [==============================] - 1s 109us/sample - loss: 4244.9815 - val_loss: 635919.6161
Epoch 7/100
7740/7740 [==============================] - 1s 110us/sample - loss: 1710.9714 - val_loss: 2334136.3464
Epoch 8/100
7740/7740 [==============================] - 1s 116us/sample - loss: 2322.3115 - val_loss: 8272069.9291
Epoch 9/100
7740/7740 [======================

RuntimeError: Cannot clone object <tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7f0edc0a1048>, as the constructor either does not set or modifies parameter learning_rate

In [10]:
rnd_search_cv.best_params_

{'learning_rate': 0.012124639227107287, 'n_hidden': 3, 'n_neurons': 67}

In [11]:
rnd_search_cv.best_score_

-0.3174608481330773